Парсинг групп ВКонтакте (код слушательницы прошлого потока, Лаврентюк Анны, в котором я убрала части, касающиеся комментариев, так как я с ними не работаю в данном проекте)

In [ ]:
!pip install vk_api
!pip install tqdm
!pip install pandas
!pip install nltk
!pip install pymystem3
!pip install wordcloud
!pip install matplotlib
!pip install scikit-learn
!pip install TextBlob

Подключаем api вконтакте для парсинга группы

In [ ]:
import vk_api
import math
import datetime
import time
import pandas as pd
from tqdm import tqdm

session = vk_api.VkApi(token='ТОКЕН')
vk = session.get_api()

Подготавливаю модели для оперирования в коде. Из-за того что данные которые возвращает методы апи имеют избыточность, то нам необходимо подготовить свои модели, для того чтобы сохранить только конкретные данные.


Группа

id - идентификатор группы int

name - имя группы на русском языке string

screen_name - имя группы на латинице

In [2]:
class vk_group:
    def __init__(self, id: int, name: str, screen_name: str):
        self.id = id
        self.name = name
        self.screen_name = screen_name


Посты группы


id - идентификатор поста int

group_id - идентификатор группы int

date - дата публикации datetime

text - текст поста в группе string

In [4]:
class vk_group_post:
    def __init__(self, id: int, group_id: int, date: datetime, text: str):
        self.id = id
        self.group_id = group_id
        self.date = date
        self.text = text

Получить информацию о группе по идентификатору

https://dev.vk.com/ru/method/groups.getById

In [5]:
def get_group_info_by_id(group_id):
    vk_group_info = None;
    try:
        group_info_from_api = vk.groups.getById(group_id=group_id, v='5.199')['groups'][0]
        vk_group_info = vk_group(group_info_from_api["id"], group_info_from_api["name"], group_info_from_api["screen_name"])
    except vk_api.exceptions.ApiError as error:
        print(f"An error occurred: {error}")

    return vk_group_info

print(vars(get_group_info_by_id(83415396)))
print(vars(get_group_info_by_id(16325867)))
print(vars(get_group_info_by_id(222935597)))

{'id': 83415396, 'name': 'Родной звук', 'screen_name': 'rodzvuk'}
{'id': 16325867, 'name': 'Zion Music', 'screen_name': 'zion_music'}
{'id': 222935597, 'name': 'ON Лейбл', 'screen_name': 'on__label'}


Получить посты в группе по её идентификатору

https://dev.vk.com/ru/method/wall.get

In [6]:
def get_posts_by_group_id(group_id, offset = 0):
    posts = []
    posts_count = 0
    try:
        response = vk.wall.get(owner_id=f'-{group_id}', count=100, offset=offset, v='5.199')
        posts_count = response['count']
        posts_from_api = response['items']
        posts = [vk_group_post(post['id'], group_id, datetime.datetime.fromtimestamp(post['date']), post['text']) for post in posts_from_api]
    except vk_api.exceptions.ApiError as error:
        print(f"An error occurred: {error}")

    return posts, posts_count

Метод сохранения списка в csv

In [7]:
def save_list_to_csv(file_name, objects):
    data = [obj.__dict__ for obj in objects]
    df = pd.DataFrame(data)
    df.to_csv(file_name, index=False, sep=';', encoding='utf-8', header=True)

Получаем все посты и сохраняем в переменную posts и сохраним в файлы csv

In [8]:
posts, posts_count = get_posts_by_group_id(83415396)

if posts_count > 0:
    current_offset = 100
    total_post_pages = math.ceil(posts_count / 100)
    for page in tqdm(range(1, total_post_pages)):
        current_posts, current_posts_count = get_posts_by_group_id(83415396, current_offset)
        posts.extend(current_posts)
        current_offset += 100
        time.sleep(1)

save_list_to_csv('posts_rz.csv', posts)

100%|██████████| 359/359 [43:29<00:00,  7.27s/it]


In [9]:
posts, posts_count = get_posts_by_group_id(16325867)

if posts_count > 0:
    current_offset = 100
    total_post_pages = math.ceil(posts_count / 100)
    for page in tqdm(range(1, total_post_pages)):
        current_posts, current_posts_count = get_posts_by_group_id(16325867, current_offset)
        posts.extend(current_posts)
        current_offset += 100
        time.sleep(1)

save_list_to_csv('posts_zion.csv', posts)

100%|██████████| 45/45 [03:34<00:00,  4.77s/it]


In [10]:
posts, posts_count = get_posts_by_group_id(222935597)

if posts_count > 0:
    current_offset = 100
    total_post_pages = math.ceil(posts_count / 100)
    for page in tqdm(range(1, total_post_pages)):
        current_posts, current_posts_count = get_posts_by_group_id(222935597, current_offset)
        posts.extend(current_posts)
        current_offset += 100
        time.sleep(1)

save_list_to_csv('posts_on.csv', posts)

100%|██████████| 8/8 [00:38<00:00,  4.87s/it]
